In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
 
X_train = pd.read_csv("X_train.csv", index_col=0)
Y_train = pd.read_csv("Y_train.csv", index_col=0)

X_train.head()

,designation,description,productid,imageid
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786


In [2]:
# FUSION DE X ET Y
# Comme nous allons probablement supprimer des lignes en nettoyant X,
# j'ajoute Y dès maintenant pour me facilité la correspondance

X_train["prdtypecode"]=Y_train["prdtypecode"]
X_train.head()

,designation,description,productid,imageid,prdtypecode
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,10
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,2280
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,50
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,1280
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,2705


In [3]:
# SUPPRESSION DES DOUBLONS DE LIGNE (lorsque designation et description sont identiques)

nblignes_avant = len(X_train)
X_train = X_train.drop_duplicates(subset=['designation', 'description'], keep='first')
nblignes_apres = len(X_train)

nbsup = nblignes_avant-nblignes_apres
print(nbsup, "lignes en doublon ont été supprimées. Il reste",nblignes_apres,"lignes")


1414 lignes en doublon ont été supprimées. Il reste 83502 lignes


In [4]:
# SUPPRESSION DES DOUBLONS EN COLONNE

# Avant de concaténer Designation et Description, on vérifie qu'il n'y a pas la même chose dans ces deux colonnes
nombre_lignes = (X_train["designation"] == X_train["description"]).sum()
print("Il y a",nombre_lignes,"lignes où designation et description contiennent les mêmes informations")

# On remplace alors la description par une chaine de caractère vide pour préparer la concaténation
X_train.loc[X_train["designation"] == X_train["description"], "description"] = ""


Il y a 179 lignes où designation et description contiennent les mêmes informations


In [5]:
# CONCATENATION DE LA COLONNE DESIGNATION ET DESCRIPTION

print("Nombre de NaN dans la colonne designation:", X_train["designation"].isna().sum())
print("Nombre de NaN dans la colonne description:", X_train["description"].isna().sum(),"\n")

X_train["description"] = X_train["description"].fillna("")     # On remplace les NaN par une chaine vide pour que la concaténation fonctionne

X_train["merged"] = X_train["designation"] + " - " + X_train["description"]          # On concatène tout dans une nouvelle colonne "merged"
X_train = X_train.drop(columns = ["designation","description"], axis =1)             # On peut alors supprimer la colonne "description"

display(X_train.head(5))


Nombre de NaN dans la colonne designation: 0
Nombre de NaN dans la colonne description: 29680 



,productid,imageid,prdtypecode,merged
0,3804725264,1263597046,10,Olivia: Personalisiertes Notizbuch / 150 Seite...
1,436067568,1008141237,2280,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...
2,201115110,938777978,50,Grand Stylet Ergonomique Bleu Gamepad Nintendo...
3,50418756,457047496,1280,Peluche Donald - Europe - Disneyland 2000 (Mar...
4,278535884,1077757786,2705,La Guerre Des Tuques - Luc a des id&eacute;es ...


In [6]:
# NOUVEAU CONTROLE SUR LES DOUBLONS DE LIGNES, SUITE A LA CONCATENATION

nblignes_avant = len(X_train)
X_train = X_train.drop_duplicates(subset=["merged"])                        # On supprime les doublons en ne gardant que la première occurance 

nblignes_apres = len(X_train)
print("Nombre de lignes supprimées :", nblignes_avant - nblignes_apres)
print("Il reste :",nblignes_apres,"lignes\n")



Nombre de lignes supprimées : 0
Il reste : 83502 lignes



In [7]:
# NORMALISATION DU TEXTE

from unidecode import unidecode

def clean_text(text):
    if isinstance(text, str):            # Si le texte est une chaîne de caractères,
        text = unidecode(text)             # Remplace les accents par des lettres sans accent
        text = text.lower()                # Met le texte en minuscule
        text = ' '.join(text.split())      # Remplace les espaces inutiles
        return text
    return text

X_train["merged"] = X_train['merged'].apply(clean_text)
display(X_train.head(40))

,productid,imageid,prdtypecode,merged
0,3804725264,1263597046,10,olivia: personalisiertes notizbuch / 150 seite...
1,436067568,1008141237,2280,journal des arts (le) ndeg 133 du 28/09/2001 -...
2,201115110,938777978,50,grand stylet ergonomique bleu gamepad nintendo...
3,50418756,457047496,1280,peluche donald - europe - disneyland 2000 (mar...
4,278535884,1077757786,2705,la guerre des tuques - luc a des id&eacute;es ...
5,5862738,393356830,2280,afrique contemporaine ndeg 212 hiver 2004 - do...
6,91920807,907794536,10,christof e: bildungsprozessen auf der spur -
7,344240059,999581347,2522,conquerant sept cahier couverture polypro 240 ...
8,4239126071,1325918866,1280,puzzle scooby-doo avec poster 2x35 pieces -
9,3793572222,1245644185,2582,tente pliante v3s5-pro pvc blanc - 3 x 4m50 - ...


In [8]:
# NOUVEAU CONTROLE SUR LES DOUBLONS DE LIGNES, SUITE A LA NORMALISATION

nblignes_avant = len(X_train)
X_train = X_train.drop_duplicates(subset=["merged"])                        # On supprime les doublons en ne gardant que la première occurance 

nblignes_apres = len(X_train)
print("Nouveaux doublons supprimés :", nblignes_avant - nblignes_apres)
print("Il reste :",nblignes_apres,"lignes\n")

Nouveaux doublons supprimés : 39
Il reste : 83463 lignes



In [9]:
# SUPPRESSION DES URLS DANS LE TEXTE

# Importation de la bibliothèque "Regular Expression"
import re

url_pattern = r'https?://\S+'
X_train["merged"] = X_train["merged"].str.replace(url_pattern, '', regex=True)


In [10]:
# TRAITEMENT DES BALISES HTML (type <b>, </p>, etc)

# Importation de la bibliothèque "Regular Expression"
import re     


# Fonction qui identifie tout ce qui est délimité par < >
def extraction_balises(texte):
    return re.findall(r"<[^>]+>", str(texte))     


# Création d'une liste d'éléments uniques
balises = set()                                   
X_train["merged"].apply(lambda x: balises.update(extraction_balises(x)))


# Création d'un dataframe et export Excel pour meilleur visibilité
df_balises = pd.DataFrame(list(balises), columns=["Balises"])
df_balises["Balises"].to_excel("fichier_balises.xlsx", index=False)

print("Nous avons identifier",len(df_balises),"balises uniques")


Nous avons identifier 953 balises uniques


En observant les textes délimités par des balises, on voit qu'il n'y a pas que des balises HTML.

Il y a également beaucoup de texte délimité par des doubles chevrons,
par exemple : << brouillard >>, <<banc des elfes>>, ou encore <<appoint electrique>>

Nous supprimons les doubles chevrons en conservant le texte

In [11]:
# SUPPRESSION DES DOUBLES CHEVRONS
X_train["merged"] = X_train["merged"].str.replace("<<", "").str.replace(">>", "")


In [12]:
# On relance le traitement des balises HTML

balises = set()                                   
X_train["merged"].apply(lambda x: balises.update(extraction_balises(x)))

df_balises = pd.DataFrame(list(balises), columns=["Balises"])
df_balises["Balises"].to_excel("fichier_balises.xlsx", index=False)

print("Il ne reste plus que",len(df_balises),"balises uniques après suppression des doubles chevrons délimitants du texte")

Il ne reste plus que 286 balises uniques après suppression des doubles chevrons délimitants du texte


In [13]:
# TRAITEMENT DES CARACTERES SPECIAUX HTML (type &amp, &quot, etc)

import html
X_train["merged"] = X_train["merged"].apply(html.unescape)

Après observation des résulats, on s'aperçoit que le nettoyage des caractères spéciaux HTML n'a pas fonctionné comme prévu.

En effet, il y a des caractères spéciaux mals encodés, par exemple "& amp;" ou "& nbsp;" avec un espace en trop et donc non reconnu.
Il y a également des caractères spéciaux HTML sans le ";", par exemple "& amp".

On ne peut pas se contenter de supprimer les espaces après le caractère & car il y a du texte tel que : "blake & mortimer".


In [14]:
# On va essayer d'identifier les cas les plus fréquents sous la forme "&" + "espace" + "un mot"

df_html = X_train["merged"].str.extract(r"(&\s\w+)")
df_html = df_html.dropna()
df_html.columns = ['caractere']

display(df_html.info())
print("\n")
display(df_html.head(10))
print("\n")
display(df_html.value_counts().head(30))

<class 'pandas.core.frame.DataFrame'>
Index: 3651 entries, 59 to 84907
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   caractere  3651 non-null   object
dtypes: object(1)
memory usage: 57.0+ KB


None

,caractere
59,& amp
62,& creme
74,& 2
96,& son
121,& nbsp
140,& amp
187,& amp
196,& r2
199,& nbsp
237,& around


caractere   
& amp           936
& nbsp          451
& lt            100
& 2              93
& gt             82
& blanc          36
& dragons        34
& play           24
& 3              22
& the            21
& easy           17
& ntsc           17
& mavic          17
& bois           16
& modeles        16
& decker         16
& d              14
& jardin         14
& smart          12
& clear          11
& stratton       11
& triphase       11
& plug           11
& zoom           10
& revetement     10
& warner         10
& 1               9
& metal           9
& noir            9
& newton          9
Name: count, dtype: int64

En affichant les 30 plus fréquents, on identifie les caractères spéciaux suivants : 
-> "&amp" qui correspond simplement à "&", on peut donc conserver le & et supprimer amp,
-> "&nbsp" qui correspond à un espace qui ne peut pas être cassé par un retour à la ligne,
-> "&lt" qui correspond à <,
-> "&gt" qui correspond à >.

Il y a plus de 3650 données identifiées. Je pense qu'on peut se contenter des 4 identifiés ci-dessus. 
Il y a peut être d'autres caractères spéciaux mais leur occurence est inférieur à 9 à l'échelle de tout le dataset.


In [15]:
X_train["merged"] = X_train["merged"].str.replace("& amp ", "&", regex=False)
X_train["merged"] = X_train["merged"].str.replace("& amp;", "&", regex=False)

X_train["merged"] = X_train["merged"].str.replace("& nbsp ", "&nbsp;", regex=False)
X_train["merged"] = X_train["merged"].str.replace("& nbsp;", "&nbsp;", regex=False)

X_train["merged"] = X_train["merged"].str.replace("& lt ", "&lt;", regex=False)
X_train["merged"] = X_train["merged"].str.replace("& lt;", "&lt;", regex=False)

X_train["merged"] = X_train["merged"].str.replace("& gt ", "&gt;", regex=False)
X_train["merged"] = X_train["merged"].str.replace("& gt;", "&gt;", regex=False)

X_train["merged"] = X_train["merged"].apply(html.unescape)

In [17]:
# Export en fichier Excel 

X_train.to_csv("fichier_nettoye.csv", index=True)
X_train.to_excel("fichier_nettoye.xlsx", index=True)